In [4]:
import json
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import requests
import time

# Load environment variables
load_dotenv()

# Get MongoDB URI from .env file
mongodb_uri = os.getenv('13F_MongoDB_URI')
apollo_key = os.getenv("13F_ApolloIo_Key")

# Connect to MongoDB
client = MongoClient(mongodb_uri)
db = client['13f_filings']
apollo_collection = db['apollo_results']

def process_linkedin_urls(company_name, linkedin_urls):
    if not linkedin_urls:
        return [{"company_name": company_name, "error": "No LinkedIn URLs found"}]

    url = "https://api.apollo.io/api/v1/people/bulk_match"
    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json',
        'X-Api-Key': apollo_key
    }

    data = {
        "reveal_personal_emails": True,
        "reveal_phone_number": True,
        "webhook_url": "https://your_webhook_site",
        "details": [{"linkedin_url": url} for url in linkedin_urls]
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return [{"company_name": company_name, "error": f"API request failed: {str(e)}"}]
    
    results = []

    api_response = response.json()
    
    for match in api_response.get('matches', []):
        result = {
            "company_name": company_name,
            "linkedin_url": match.get('linkedin_url'),
            "email": match.get('email'),
            "first_name": match.get('first_name'),
            "last_name": match.get('last_name'),
            "title": match.get('title'),
            "current_employer": match.get('current_employer'),
            "company_linkedin_url": match.get('company_linkedin_url')
        }
        results.append(result)

    return results if results else [{"company_name": company_name, "error": "No matches found"}]

def main():
    # Read the LinkedIn URLs from the JSON file
    with open('all_companies_linkedin_links.json', 'r') as f:
        linkedin_data = json.load(f)

    all_apollo_results = []

    for company_name, linkedin_urls in linkedin_data.items():
        print(f"Processing {company_name}")
        company_results = process_linkedin_urls(company_name, linkedin_urls)
        all_apollo_results.extend(company_results)
        
        # Add a delay to respect API rate limits
        time.sleep(1)

    # Save Apollo results to a JSON file
    with open('apollo_results.json', 'w') as f:
        json.dump(all_apollo_results, f, indent=2)

    print("Apollo results saved to apollo_results.json")

    # Push Apollo results to MongoDB
    apollo_collection.insert_many(all_apollo_results)

    print("Results pushed to MongoDB")

    # Close the MongoDB connection
    client.close()

if __name__ == "__main__":
    main()

Processing Creekside Partners
Processing Gordian Capital Singapore Pte Ltd
Processing Brown Financial Advisors
Processing NCP Inc.
Processing SW Investment Management LLC
Processing Centennial Bank AR
Processing Mendon Capital Advisors Corp
Processing Pensioenfonds Rail & OV
Processing Strategic Investment Solutions Inc. IL
Processing University of Texas Texas AM Investment Management Co.
Processing SRN Advisors LLC
Processing Cowen Investment Management LLC
Processing Maven Securities LTD
Processing Gibson Wealth Advisors LLC
Processing RiverFront Investment Group LLC
Processing Heritage Oak Wealth Advisors LLC
Processing Kestra Investment Management LLC
Processing Sycomore Asset Management
Processing Birmingham Capital Management Co. Inc. AL
Processing Farmers National Bank
Processing Souders Financial Advisors
Processing Logos Global Management LP
Processing Clearbridge Investments LLC
Processing Carrera Capital Advisors
Processing New Wave Wealth Advisors LLC
Processing Dana Invest

In [12]:
with open('all_companies_linkedin_links.json', 'r') as f:
    linkedin_data = json.load(f)

def testing_dataPayload(linkedin_urls):
    data = {
        "reveal_personal_emails": True,
        "reveal_phone_number": True,
        "webhook_url": "https://your_webhook_site",
        "details": [{"linkedin_url": url} for url in linkedin_urls]
    }
    print(data)

for company_name, linkedin_urls in linkedin_data.items():
    print(f"Processing {company_name}\n")
    testing_dataPayload(linkedin_urls)

Processing Creekside Partners

{'reveal_personal_emails': True, 'reveal_phone_number': True, 'webhook_url': 'https://your_webhook_site', 'details': [{'linkedin_url': 'https://www.linkedin.com/in/teresa-coleman-9157758'}, {'linkedin_url': 'https://www.linkedin.com/in/michael-gorman-b6a19020'}, {'linkedin_url': 'https://www.linkedin.com/in/jesse-ashford-28287027a'}]}
Processing Gordian Capital Singapore Pte Ltd

{'reveal_personal_emails': True, 'reveal_phone_number': True, 'webhook_url': 'https://your_webhook_site', 'details': []}
Processing Brown Financial Advisors

{'reveal_personal_emails': True, 'reveal_phone_number': True, 'webhook_url': 'https://your_webhook_site', 'details': [{'linkedin_url': 'https://www.linkedin.com/in/jeffrey-brown-7a64805a'}, {'linkedin_url': 'https://www.linkedin.com/in/gregory-a-brown-14685114'}, {'linkedin_url': 'https://www.linkedin.com/in/clintrbrown'}, {'linkedin_url': 'https://www.linkedin.com/in/tonya-brown-20b09420'}, {'linkedin_url': 'https://www.lin